In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import glob
import pprint

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from collections import OrderedDict

from adapters import *
from dataset import NumpyCsvDataSet
from runner import ExptRunner
from networks.imageencoder import *
from networks.imagedecoder import *
from networks.DenseAutoEncoder import DenseAutoEncoder
from networks.composedautoencoder import ComposedAutoEncoder
from networks.ConvVae import ConvVae
from networks.vae import VAE
from networks.dense import *
from networks.lossfunctions import *
from networks.multinet import *
from networks.special import *
from helpers import ReportResults
from helpers import *
import utils

In [3]:
opt = np.get_printoptions()
np.set_printoptions(threshold=np.inf)

In [4]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print (device)

cpu


In [5]:
start = time.time()
data_folder = 'data/demoplanner/version1'
v1_train_data = NumpyCsvDataSet(data_folder + '/train.csv', device=device)
v1_test_data = NumpyCsvDataSet(data_folder + '/test.csv', device=device)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 26.74958372116089 sec


In [5]:
start = time.time()
data_folder = 'data/demoplanner/version2'
v2_train_data = NumpyCsvDataSet(data_folder + '/train.csv', device=device)
v2_test_data = NumpyCsvDataSet(data_folder + '/test.csv', device=device)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 12.131418943405151 sec


In [6]:
train_data = v1_train_data
test_data = v1_test_data

In [7]:
print (train_data.data.shape)
print (test_data.data.shape)

print (train_data.data.device)
print (test_data.data.device)

print (train_data.data.dtype)
print (test_data.data.dtype)

torch.Size([13280, 2058])
torch.Size([1341, 2058])
cpu
cpu
torch.float32
torch.float32


In [ ]:
pretrain_policy = Dense([4, 16, 16, 6], use_last_act=False, prefix='enc').to(device)
pretrain_policy.load_state_dict(torch.load('runs/pretrained/04-08-10-24-50-Policy-V2-Dense-CE/autoenc.pth', 
                                  map_location=device))

pretrain_dynamics = Dense([4, 16, 2], prefix='enc').to(device)
pretrain_dynamics.load_state_dict(torch.load('runs/pretrained/03-31-18-02-52-Dynamics-V1-Dense-SmoothL1/autoenc.pth', 
                                    map_location=device))

pretrain_imgEncDec = ComposedAutoEncoder(layers_channels=[16,16,16,16], useMaxPool=True, device=device).to(device)
pretrain_imgEncDec.load_state_dict(torch.load('runs/pretrained/03-28-14-10-20-DemoPl-V2-ComposedAutoEncoderWithMaxPool-SmoothL1-/autoenc.pth',
                               map_location=device))

pretrain_envEncDec_chkpt = torch.load(
    'runs/pretrained/04-20-11-03-59-EnvAutoEnc-V2-DenseAutoEncoder-MSE/train_checkpoint.tar',map_location=device)
pretrain_envEncDec = pretrain_envEncDec_chkpt['model']

pretrain_ImgToEnvBaseLine_chkpt = torch.load(
    'runs/pretrained/04-21-12-51-48-ImageToEnvFunctionBaseLine-V2-3Conv2Dense-MSE/train_checkpoint.tar',map_location=device)
pretrain_ImgToEnvBaseLine = pretrain_ImgToEnvBaseLine_chkpt['model']

In [104]:
net_builder = lambda: MultiNet(policy=Dense([4, 16, 16, 6], use_last_act=False),
                               dynamics=Dense([4, 16, 2]))

In [13]:
img_layers = [16,16,16,16]
z_dim = 64 # This should be set based on above img_layers
env_layers = [4, z_dim]
in_channels = 1
img_encoder = lambda: ImageEncoderFlatInput(in_channels, img_layers, 'imgenc', useMaxPool=True, addFlatten=True)
img_decoder = lambda: ImageDecoderFlatInput(z_dim, list(reversed(img_layers)), in_channels, 'imgdec', useSigmoid=True)
env_encoder = lambda: Dense(env_layers, use_last_act=False, prefix='envenc')
env_decoder = lambda: Dense(list(reversed(env_layers)), last_act='sigmoid', prefix='envdec')
combined = lambda l1, l2: nn.Sequential(OrderedDict([
                              ('EncNet', l1()),
                              ('DecNet', l2()),
                            ]))

In [9]:
def env_enc_pretrained_img_dec():
    net = combined(env_encoder, img_decoder).to(device)
    load_mapped_state_dict(net, pretrain_imgEncDec, 'DecNet.net.imgdec', 'decoder.net.dec')
    return net

def pretrained_img_enc_env_dec():
    net = combined(img_encoder, env_decoder).to(device)
    load_mapped_state_dict(net, pretrain_imgEncDec, 'EncNet.net.imgenc', 'encoder.net.enc')
    return net

def img_enc_env_dec_from_pretrained():
    net = combined(img_encoder, env_decoder).to(device)
    load_mapped_state_dict(net, pretrain_imgEncDec, 'EncNet.net.imgenc', 'encoder.net.enc')
    load_mapped_state_dict(net, pretrain_envEncDec, 'DecNet.net.envdec_fc0', 'decoder.dec_fc1') # Hacky
    return net

def multinet_with_pretrained_policy_and_dynamics():
    net = MultiNet(policy=Dense([4, 16, 16, 6], use_last_act=False, prefix='enc'),
                   dynamics=Dense([4, 16, 2], prefix='enc')).to(device)
    net.policy.load_state_dict(pretrain_policy.state_dict())
    net.dynamics.load_state_dict(pretrain_dynamics.state_dict())
    return net

def image_to_env(conv, dense):
    return nn.Sequential(OrderedDict([
        ('Conv', conv),
        ('Dense', dense)
    ])).to(device)

def end_to_end_net():
    ItoY = image_to_env(ImageEncoderFlatInput(1, [16,16,16], prefix='imgenc', useMaxPool=True, addFlatten=True),
                     Dense([256, 4]))
    policy = Dense([4, 16, 16, 6], use_last_act=False, prefix='pol')
    dynamics = Dense([4, 16, 2], prefix='dyn')
    net = EndToEndNet(ItoY=ItoY, policy=policy, dynamics=dynamics)
    return net

def end_to_end_net_from_pretrained():
    ItoY = image_to_env(ImageEncoderFlatInput(1, [16,16,16], prefix='imgenc', useMaxPool=True, addFlatten=True),
                     Dense([256, 4]))
    policy = Dense([4, 16, 16, 6], use_last_act=False, prefix='pol')
    dynamics = Dense([4, 16, 2], prefix='dyn')
    net = EndToEndNet(ItoY=ItoY, policy=policy, dynamics=dynamics)
    load_mapped_state_dict(net, pretrain_policy, 'policy.net.pol', 'net.enc')
    load_mapped_state_dict(net, pretrain_dynamics, 'dynamics.net.dyn', 'net.enc')
    load_mapped_state_dict(net, pretrain_ImgToEnvBaseLine, 'ItoY.Conv', 'Conv')
    load_mapped_state_dict(net, pretrain_ImgToEnvBaseLine, 'ItoY.Dense', 'Dense')
    return net

In [44]:
def run_expts(train_epochs):
    for i in range(len(expts)):
        arg_lists = expts[i]
        cons_args = arg_lists[0]
        train_args = {}
        if (len(arg_lists)) > 1:
            train_args = arg_lists[1]
        test_args = {}
        if (len(arg_lists)) > 2:
            test_args = arg_lists[2]
        
        runner = ExptRunner(train_data=train_data, test_data=test_data, device=device, **cons_args)
        print ("Experiment logs folder: {}".format(runner.log_folder))
        with open('runs/' + runner.log_folder + '/expt.txt', 'w') as f:
            f.write(pprint.pformat(arg_lists))
        runner.train(train_epochs, **train_args)
        runner.test(**test_args)

In [71]:
expts = [
    [{
        "expt_prefix":'FineTunePreTrainedEndToEnd-V2-EndToEndEnv-MSE',
        "net_func": end_to_end_net_from_pretrained,
        "data_adapter_func": identity_adapter,
        "loss_adapter_func": EndToEndEnv_EndToEnd_mse_loss_adapter,
     },
    {},
    {
        "loss_adapter": EndToEndEnv_EndToEnd_l1_loss_adapter
    }],
    [{
        "expt_prefix":'FineTunePreTrainedEndToEnd-V2-EndToEndEnv-L1',
        "net_func": end_to_end_net_from_pretrained,
        "data_adapter_func": identity_adapter,
        "loss_adapter_func": EndToEndEnv_EndToEnd_l1_loss_adapter,
     },
    {},
    {
        "loss_adapter": EndToEndEnv_EndToEnd_l1_loss_adapter
    }],
    [{
        "expt_prefix":'TrainEndToEnd-V2-EndToEndEnv-MSE',
        "net_func": end_to_end_net,
        "data_adapter_func": identity_adapter,
        "loss_adapter_func": EndToEndEnv_EndToEnd_mse_loss_adapter,
     },
    {},
    {
        "loss_adapter": EndToEndEnv_EndToEnd_l1_loss_adapter
    }],
    [{
        "expt_prefix":'TrainEndToEnd-V2-EndToEndEnv-L1',
        "net_func": end_to_end_net,
        "data_adapter_func": identity_adapter,
        "loss_adapter_func": EndToEndEnv_EndToEnd_l1_loss_adapter,
     },
    {},
    {
        "loss_adapter": EndToEndEnv_EndToEnd_l1_loss_adapter
    }],
]

In [72]:
train_epochs = 2
run_expts(train_epochs)

Experiment logs folder: runs/05-04-19-12-55-FineTunePreTrainedEndToEnd-V2-EndToEndEnv-MSE
Experiment logs folder: runs/05-04-19-13-43-FineTunePreTrainedEndToEnd-V2-EndToEndEnv-L1


In [47]:
runner = ExptRunner('ForTestReport', net_func= lambda:net, train_data=train_data, test_data=test_data, device=device,
                    loss_adapter_func=None, data_adapter_func=chkpt['input_adapter'], data_to_label_adapter=chkpt['label_adapter'])
print ("Experiment logs folder: {}".format(runner.log_folder))
runner.test()

Experiment logs folder: runs/04-28-14-26-28-ForTestReport


In [ ]:
def build_reporter(checkpoint_file, useL1Loss=True):
    rr = ReportResults(test_data.data, train_data.data, device,
                       checkpoint_file, useL1Loss)
    return rr

In [8]:
def combined_net_report(checkpoint_file):
    cp = torch.load(checkpoint_file, map_location=device)
    net = cp['model']
    se = SpecialExptRunner('', net, None, test_data, device=device)
    se.eval_end_of_training()

In [32]:
sample = torch.cat((test_data.data[1055:1060], test_data.data[1230:1235]), axis=0)
ground_truth = policy_groud_truth_class_adapter(sample).to(device)
for folder, net in rr.build_net(rootdir='runs/04-07/'):
    print ('Folder: ' + folder)
    op_batch, loss = rr.run_mini_batch(net, sample)
    x_loss = F.cross_entropy(op_batch[:, :3], ground_truth[:, 0], reduction='none')
    y_loss = F.cross_entropy(op_batch[:, 3:], ground_truth[:, 1], reduction='none')
    print (ground_truth)
    print (op_batch)
    print (loss)
    print (x_loss)
    print (y_loss)
    print ('-----------------------------------------')

Folder: 04-07-17-58-07-Policy-V1-Dense-CE
tensor([[0, 1],
        [0, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [2, 0],
        [2, 0],
        [2, 0],
        [2, 0],
        [2, 1]])
tensor([[9.9892e-01, 1.1768e-02, 4.4407e-06, 6.9097e-01, 4.3719e-08, 5.8148e-05],
        [9.8408e-01, 4.1556e-02, 1.0096e-03, 9.8178e-01, 5.3358e-06, 1.9832e-04],
        [1.0000e+00, 1.0968e-05, 2.3005e-34, 5.3218e-30, 2.1840e-32, 2.9180e-05],
        [1.0000e+00, 4.4320e-05, 4.1472e-32, 7.9823e-29, 2.2644e-30, 4.3043e-04],
        [9.9999e-01, 1.9323e-04, 6.4159e-30, 1.2560e-27, 2.0004e-28, 5.3637e-03],
        [2.7807e-11, 4.0247e-02, 9.9016e-01, 9.9007e-01, 3.9920e-02, 2.1634e-12],
        [1.2382e-08, 3.5409e-02, 9.9082e-01, 9.9139e-01, 3.6082e-02, 8.6834e-10],
        [4.4673e-06, 3.4880e-02, 9.9111e-01, 9.9073e-01, 3.2762e-02, 3.6731e-07],
        [1.6327e-03, 3.4472e-02, 9.9060e-01, 9.8911e-01, 2.6565e-02, 1.5856e-04],
        [0.0000e+00, 7.8143e-03, 9.9971e-01, 0.0000e+00, 9.

In [21]:
sample = torch.cat((test_data.data[1055:1060], test_data.data[1230:1235]), axis=0)
ground_truth = policy_groud_truth_adapter(sample).to(device)
for folder, net in rr.build_net(rootdir='runs/04-08/'):
    print ('Folder: ' + folder)
    op_batch, loss = rr.run_mini_batch(net, sample)
    print (ground_truth)
    print (op_batch)
    print (loss)

Folder: 04-08-07-35-53-Policy-V2-Dense-MSE
tensor([[-1.,  0.],
        [-1., -1.],
        [-1.,  0.],
        [-1.,  0.],
        [-1.,  0.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1.,  0.]])
tensor([[-1.0000e+00, -5.9831e-04],
        [-9.9837e-01, -9.9866e-01],
        [-1.0000e+00, -2.0846e-03],
        [-1.0000e+00, -2.0014e-03],
        [-1.0000e+00, -1.9175e-03],
        [ 9.9764e-01, -9.9867e-01],
        [ 9.9765e-01, -9.9868e-01],
        [ 9.9766e-01, -9.9868e-01],
        [ 9.9767e-01, -9.9868e-01],
        [ 1.0000e+00,  7.1299e-04]], grad_fn=<TanhBackward>)
tensor(0.0012, grad_fn=<MeanBackward0>)
Folder: 04-08-07-29-36-Policy-V2-Dense-MSE
tensor([[-1.,  0.],
        [-1., -1.],
        [-1.,  0.],
        [-1.,  0.],
        [-1.,  0.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1.,  0.]])
tensor([[-0.9994, -0.0311],
        [-0.9682, -0.9674],
        [-0.9995,  0.0176],
   

In [ ]:
np.set_printoptions(**opt)